In [1]:
(member :sb-thread *features*)

(:SB-THREAD :SB-UNICODE :SBCL :WIN32)

# Threading bascis

In [2]:
(sb-thread:list-all-threads)

(#<SB-THREAD:THREAD tid=21608 "Anonymous thread" RUNNING {11057100A3}>
 #<SB-THREAD:THREAD tid=23488 "SHELL Thread" RUNNING {11044723E3}>
 #<SB-THREAD:THREAD tid=13668 "main thread" RUNNING {1100BB8003}>)

In [1]:
(import 'sb-thread:make-thread)

(let ()
  (print (sb-thread:list-all-threads))
  (let ((t1 (make-thread (lambda ()
                           (print "Hello, thread.")
                           'ok)
                           :name "hello-t")))
    (print (sb-thread:list-all-threads))
    (print (sb-thread:join-thread t1)))
  (print (sb-thread:list-all-threads)))


T

(#<SB-THREAD:THREAD tid=21608 "Anonymous thread" RUNNING {11057100A3}>
 #<SB-THREAD:THREAD tid=23488 "SHELL Thread" RUNNING {11044723E3}>
 #<SB-THREAD:THREAD tid=13668 "main thread" RUNNING {1100BB8003}>)


(#<SB-THREAD:THREAD tid=21608 "Anonymous thread" RUNNING {11057100A3}>
 #<SB-THREAD:THREAD tid=23488 "SHELL Thread" RUNNING {11044723E3}>
 #<SB-THREAD:THREAD tid=13668 "main thread" RUNNING {1100BB8003}>) 
(#<SB-THREAD:THREAD tid=26552 "hello-t" RUNNING {1101B2CF13}>
 #<SB-THREAD:THREAD tid=21608 "Anonymous thread" RUNNING {11057100A3}>
 #<SB-THREAD:THREAD tid=23488 "SHELL Thread" RUNNING {11044723E3}>
 #<SB-THREAD:THREAD tid=13668 "main thread" RUNNING {1100BB8003}>) 
OK 
(#<SB-THREAD:THREAD tid=21608 "Anonymous thread" RUNNING {11057100A3}>
 #<SB-THREAD:THREAD tid=23488 "SHELL Thread" RUNNING {11044723E3}>
 #<SB-THREAD:THREAD tid=13668 "main thread" RUNNING {1100BB8003}>) 

# Special Variables

In [ ]:
(defparameter *x* 0)
(let ((*x* 1))
  (sb-thread:join-thread
    ; threads do NOT inherit dynamic bindings from the parent thread
    (sb-thread:make-thread (lambda () *x*))))

*X*

0

# Atomic Operations

In [2]:
(defparameter *c* (cons 0 nil))
(defun make-counter (name)
  (sb-thread:make-thread (lambda ()
                           (dotimes (idx 10000 idx)
                             (sb-ext:atomic-incf (car *c*))))
                         :name name))

(let ((t1 (make-counter "t1"))
      (t2 (make-counter "t2")))
  (+ (sb-thread:join-thread t1)
     (sb-thread:join-thread t2)))

*C*

MAKE-COUNTER

20000

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::MAKE-COUNTER in DEFUN


In [ ]:
(macro-function 'sb-ext:compare-and-swap)
(macro-function 'sb-ext:cas)

(describe 'sb-ext:cas)

#<FUNCTION (MACRO-FUNCTION COMPARE-AND-SWAP) {10007EBFDB}>

#<FUNCTION (MACRO-FUNCTION CAS) {10007EBB4B}>


SB-EXT:CAS
  [symbol]

CAS names a macro:
  Lambda-list: (PLACE OLD NEW)
  Documentation:
    Synonym for COMPARE-AND-SWAP.
    
    Additionally DEFUN, DEFGENERIC, DEFMETHOD, FLET, and LABELS can be also used to
    define CAS-functions analogously to SETF-functions:
    
      (defvar *foo* nil)
    
      (defun (cas foo) (old new)
        (cas (symbol-value '*foo*) old new))
    
    First argument of a CAS function is the expected old value, and the second
    argument of is the new value. Note that the system provides no automatic
    atomicity for CAS functions, nor can it verify that they are atomic: it is up
    to the implementor of a CAS function to ensure its atomicity.
    
    EXPERIMENTAL: Interface subject to change.
  Source file: SYS:SRC;CODE;MACROS.LISP


# Mutex Support

In [ ]:
(defvar *a-mutex* (sb-thread:make-mutex :name "my lock"))
; with output lock for multiple threads to dumps to console
(defvar *log-mutex* (sb-thread:make-mutex :name "log lock"))

(defun make-acquire (name)
  (let ((t-l *standard-output*)) ; no dynamic bindings
    (sb-thread:make-thread
      (lambda ()
        (sb-thread:with-mutex (*log-mutex*)
          (format t-l "Thread ~A running ~%" sb-thread:*current-thread*))
        (sb-thread:with-mutex (*a-mutex*)
          (sb-thread:with-mutex (*log-mutex*)
            (format t-l "Thread ~A got the lock~%" sb-thread:*current-thread*))
          (sleep (random 2)))
        (sb-thread:with-mutex (*log-mutex*)
          (format t-l "Thread ~A droped lock, dyning now~%" sb-thread:*current-thread*)))
      :name name)))

(let ((t1 (make-acquire "t1"))
      (t2 (make-acquire "t2")))
  (list (sb-thread:join-thread t1)
        (sb-thread:join-thread t2)))

*A-MUTEX*

*LOG-MUTEX*

MAKE-ACQUIRE

(NIL NIL)

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::MAKE-ACQUIRE in DEFUN
Thread #<THREAD tid=20712 "t1" RUNNING {11023EDBB3}> running 
Thread #<THREAD tid=20712 "t1" RUNNING {11023EDBB3}> got the lock
Thread #<THREAD tid=10272 "t2" RUNNING {11023EDD53}> running 
Thread #<THREAD tid=20712 "t1" RUNNING {11023EDBB3}> droped lock, dyning now
Thread #<THREAD tid=10272 "t2" RUNNING {11023EDD53}> got the lock
Thread #<THREAD tid=10272 "t2" RUNNING {11023EDD53}> droped lock, dyning now


In [ ]:
(format jupyter:*html-output* "~A~%" 123)

NIL

123

In [7]:
(defun print-message-top-level-fixed ()
  (let ((top-level *standard-output*))
    (sb-thread:make-thread
      (lambda ()
        (format top-level "Hello from thread!"))))
  nil)

(print-message-top-level-fixed)

PRINT-MESSAGE-TOP-LEVEL-FIXED

NIL

Hello from thread!

# Semaphores

# Waitqueue/condition variables

# Barriers

# Sessions/Debugging

# Foreign threads